<a href="https://colab.research.google.com/github/RobBurnap/Bioinformatics-MICR4203-MICR5203/blob/main/Bioinformatics-MICR4203-MICR5203/notebooks/02_Sequence_Alignment/Diverse_TaxID_Class_Level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install FIRST, then import
%pip install -q biopython       # Install the Biopython package quietly (-q suppresses most output) so we can work with biological sequence files

from google.colab import drive  # Import the module that lets Colab interact with Google Drive
drive.mount('/content/drive', force_remount=True)   # Mount your Google Drive so it appears in Colab's file system under /content/drive

import os, pandas as pd          # Import 'os' for file/directory operations, and pandas for working with data tables
from Bio import SeqIO            # Import SeqIO from Biopython for reading/writing biological sequence files (FASTA, GenBank, etc.)
import matplotlib.pyplot as plt  # Import Matplotlib's plotting library to create figures and graphs

print("✅ Dependencies installed & Drive mounted.")

ValueError: Mountpoint must not already contain files

In [ ]:
# ─── Setup for Colab Environment with COURSE Paths ─────
from Bio import Entrez
from pathlib import Path
from collections import defaultdict
import time, random

# ─── Required Entrez Email ─────
Entrez.email = "your_email@university.edu"  # ← Replace with your real email

# ─── User-defined Paths ─────
COURSE_DIR    = "/content/drive/MyDrive/Teaching/BIOINFO4-5203-F25"
LECTURE_CODE  = "L02-BLASTp"
TOPIC         = "diversity"

DATA_DIR   = Path(f"{COURSE_DIR}/Data/{LECTURE_CODE}_{TOPIC}/Data")
OUTPUT_DIR = Path(f"{COURSE_DIR}/Outputs/{LECTURE_CODE}_{TOPIC}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("📂 COURSE_DIR :", COURSE_DIR)
print("📁 DATA_DIR   :", DATA_DIR)
print("📁 OUTPUT_DIR :", OUTPUT_DIR)

# ─── Parameters You Can Adjust ─────
HIGHER_TAXID = "40674"         # E.g., Mammalia
MAX_TAXIDS = 500               # Number of species-level TaxIDs
ONE_PER_GENUS = True           # Enforce 1 species per genus
SLEEP_BETWEEN_CALLS = 0.4      # Politeness delay for NCBI

# ─── Output File in OUTPUT_DIR ─────
OUTPUT_FILE = OUTPUT_DIR / f"taxids-{HIGHER_TAXID}_diverse.txt"

# ─── Fetch NCBI Lineage Info ─────
def get_tax_lineage(taxid):
    try:
        handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
        records = Entrez.read(handle)
        return records[0].get("LineageEx", []), records[0].get("ScientificName", "")
    except Exception as e:
        print(f"[warn] TaxID {taxid} lineage failed: {e}")
        return [], None

# ─── Step 1: Fetch All Species Under Class ─────
print(f"📥 Fetching species under TaxID {HIGHER_TAXID}...")

search_handle = Entrez.esearch(
    db="taxonomy",
    term=f"txid{HIGHER_TAXID}[Subtree] AND species[Rank]",
    retmax=5000
)
search_results = Entrez.read(search_handle)
all_taxids = search_results["IdList"]
print(f"✅ Found {len(all_taxids)} species-level TaxIDs under this Class.")

# ─── Step 2: Filter for Genus Diversity ─────
kept = []
seen_genera = set()

for i, tid in enumerate(all_taxids):
    lineage, name = get_tax_lineage(tid)
    genus = None
    for rank in lineage:
        if rank["Rank"] == "genus":
            genus = rank["ScientificName"]
            break
    if not genus:
        continue
    if ONE_PER_GENUS and genus in seen_genera:
        continue
    seen_genera.add(genus)
    kept.append((tid, genus, name))
    if len(kept) >= MAX_TAXIDS:
        break
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Selected {len(kept)} taxids across {len(seen_genera)} genera.")

# ─── Step 3: Write Output ─────
with open(OUTPUT_FILE, "w") as f:
    for tid, genus, name in kept:
        f.write(f"{tid}	{genus}	{name}\n")

print(f"💾 Saved taxid list to: {OUTPUT_FILE}")

In [ ]:
!pip install biopython